In [1]:
import pymongo
from pymongo import MongoClient
import re

client = MongoClient('localhost', 27017)

db = client.Blocket

db.collection_names()

# db.cars.copyTo('carsClean')
cars = db.carsClean

In [2]:
cars.find_one()

{u'_id': ObjectId('574521095c72dbf51d28efdd'),
 u'brand': u'Ford',
 u'city': u'Lund',
 u'color': u'Gr\xe5',
 u'county': u'Lund',
 u'date_announced': u'2016-05-24 23:42:00',
 u'date_collected': u'2016-05-25 05:50:33.389419',
 u'description': u'SVENSKS\xc5LD FORD MONDEO 2009, en \xe4gare, s\xe4ljes pga.\nbyte till tj\xe4nstebil. Bensin/etanol, elstol\nf\xf6rare, fullst\xe4mplad servicebok, AUX-ing\xe5ng,\nABS-bromsar, ACC/klimatanl\xe4ggning, airbag f\xf6rare &\npassagerare, antisladdsystem, CD-stereo, elhissar\nfram och bak, elspeglar, farth\xe5llare, fj\xe4rrstyrt\nc-l\xe5s, f\xe4rddator, multifunktionsratt,\nservostyrning, sidoairbag, stolv\xe4rme fram,\nyttertemperaturm\xe4tareNya sommard\xe4ck och vinterd\xe4ck medf\xf6ljer. Ny\nvindruta vintern 2015/2016. Lastgaller ing\xe5r.',
 u'emissions': None,
 u'fuel': u'Milj\xf6br\xe4nsle/Hybrid',
 u'gearbox': u'Manuell',
 u'horse_power': 145,
 u'in_traffic': u'2009-04-24',
 u'is_available': True,
 u'mileage': u'13000-13499',
 u'model': u'BA

In [3]:
cars.distinct('brand')

[u'Ford',
 u'Volkswagen',
 u'Volvo',
 u'Toyota',
 u'Peugeot',
 u'Nissan',
 u'Audi',
 u'Mercedes-Benz',
 u'Opel',
 u'Honda',
 u'Ssang Yong',
 u'Hyundai',
 u'BMW',
 u'Kia',
 u'Mini',
 u'Saab',
 u'Renault',
 u'Land Rover',
 u'Skoda',
 u'Mitsubishi',
 u'Chevrolet',
 u'Mazda',
 u'Fiat',
 u'Jeep',
 u'Citro\xebn',
 u'Dacia',
 u'Lexus',
 u'SEAT',
 u'Subaru',
 u'Rover/BMC',
 u'Pontiac',
 u'Jaguar',
 u'Dodge',
 u'Cadillac',
 u'Alfa Romeo',
 u'Chrysler',
 u'\xd6vriga',
 u'Porsche',
 u'Suzuki',
 u'Smart',
 u'Lincoln',
 u'Maserati']

In [4]:
# remove None
# result = cars.delete_many({'brand': None})
# result.deleted_count
cars.find({'brand': None}).count()

0

In [5]:
# cars.find().count()

In [6]:
cars.distinct('gearbox')

[u'Manuell', u'Automat']

In [7]:
cars.find({'gearbox': '-'}).count()

0

In [8]:
result = cars.delete_many({'gearbox': '-'})
result.deleted_count

0

In [9]:
cars.distinct('fuel')

[u'Milj\xf6br\xe4nsle/Hybrid', u'Diesel', u'Bensin']

In [10]:
cars.find({'fuel': u'-'}).count()

0

In [11]:
# cars.distinct('mileage')

In [12]:
pattern = re.compile(r'-')
mileage_cats_list = cars.find({'mileage': {'$regex': pattern}}).distinct('mileage')

In [13]:
len(mileage_cats_list)

64

In [14]:
mileage_cats = map(
        lambda x: (x.split('-')),
        mileage_cats_list
)
mileage_cats = filter(None, map(lambda lst: filter(None, lst), mileage_cats))
    
mileage_cats = map(
    lambda(x, y): (int(x.replace(' ', '')), int(y.replace(' ', ''))),
    mileage_cats
)

In [15]:
mileage_cats_sorted = sorted(mileage_cats, key=lambda (x, y): x)

In [16]:
mileage_range_to_cat_dict = {(low, high): "%s-%s" %(low, high)  for (low, high) in mileage_cats_sorted}

In [17]:
cars.find({'mileage': u'Mer\xe4n50000'}).count()

0

In [19]:
# car = cars.find_one()
# mil =  int(car['mileage'].split('-')[0].replace(' ', ''))
# for rng in mileage_range_to_cat_dict.keys():
#     if rng[0] <= mil < rng[1]:
#         print rng, mil,  mileage_range_to_cat_dict[rng], car['mileage']
def clean_mileage():
    i = 0 
    for car in cars.find():
        try:
            mil =  int(car['mileage'].split('-')[0].replace(' ', ''))
        except:
            cars.delete_one({'_id': car['_id']})
        else:
            cleaned = False
            for rng in mileage_range_to_cat_dict.keys():
                if rng[0] <= mil < rng[1]:
                    cars.update_one(
                        {'_id': car['_id'] },
                        {
                          '$set': {
                            'mileage': mileage_range_to_cat_dict[rng]
                          }
                        }, upsert=False)
                    cleaned = True
                    break
            if not cleaned:   
                cars.delete_one({'_id': car['_id']})
        

In [135]:
# for rng in mileage_range_to_cat_dict.keys():
#     print mileage_range_to_cat_dict[rng]

In [20]:
clean_mileage()

In [21]:
cars.find_one()

{u'_id': ObjectId('574521095c72dbf51d28efdd'),
 u'brand': u'Ford',
 u'city': u'Lund',
 u'color': u'Gr\xe5',
 u'county': u'Lund',
 u'date_announced': u'2016-05-24 23:42:00',
 u'date_collected': u'2016-05-25 05:50:33.389419',
 u'description': u'SVENSKS\xc5LD FORD MONDEO 2009, en \xe4gare, s\xe4ljes pga.\nbyte till tj\xe4nstebil. Bensin/etanol, elstol\nf\xf6rare, fullst\xe4mplad servicebok, AUX-ing\xe5ng,\nABS-bromsar, ACC/klimatanl\xe4ggning, airbag f\xf6rare &\npassagerare, antisladdsystem, CD-stereo, elhissar\nfram och bak, elspeglar, farth\xe5llare, fj\xe4rrstyrt\nc-l\xe5s, f\xe4rddator, multifunktionsratt,\nservostyrning, sidoairbag, stolv\xe4rme fram,\nyttertemperaturm\xe4tareNya sommard\xe4ck och vinterd\xe4ck medf\xf6ljer. Ny\nvindruta vintern 2015/2016. Lastgaller ing\xe5r.',
 u'emissions': None,
 u'fuel': u'Milj\xf6br\xe4nsle/Hybrid',
 u'gearbox': u'Manuell',
 u'horse_power': 145,
 u'in_traffic': u'2009-04-24',
 u'is_available': True,
 u'mileage': u'13000-13499',
 u'model': u'BA

In [22]:
len(cars.distinct('mileage')), cars.count()

(64, 17586)

In [23]:
hp = sorted(cars.distinct('horse_power'))

In [24]:
cars.delete_many({'horse_power': None})

In [25]:
# cars.find({'horse_power': None}).count()

In [26]:
cars.find_one()

{u'_id': ObjectId('574521095c72dbf51d28efdd'),
 u'brand': u'Ford',
 u'city': u'Lund',
 u'color': u'Gr\xe5',
 u'county': u'Lund',
 u'date_announced': u'2016-05-24 23:42:00',
 u'date_collected': u'2016-05-25 05:50:33.389419',
 u'description': u'SVENSKS\xc5LD FORD MONDEO 2009, en \xe4gare, s\xe4ljes pga.\nbyte till tj\xe4nstebil. Bensin/etanol, elstol\nf\xf6rare, fullst\xe4mplad servicebok, AUX-ing\xe5ng,\nABS-bromsar, ACC/klimatanl\xe4ggning, airbag f\xf6rare &\npassagerare, antisladdsystem, CD-stereo, elhissar\nfram och bak, elspeglar, farth\xe5llare, fj\xe4rrstyrt\nc-l\xe5s, f\xe4rddator, multifunktionsratt,\nservostyrning, sidoairbag, stolv\xe4rme fram,\nyttertemperaturm\xe4tareNya sommard\xe4ck och vinterd\xe4ck medf\xf6ljer. Ny\nvindruta vintern 2015/2016. Lastgaller ing\xe5r.',
 u'emissions': None,
 u'fuel': u'Milj\xf6br\xe4nsle/Hybrid',
 u'gearbox': u'Manuell',
 u'horse_power': 145,
 u'in_traffic': u'2009-04-24',
 u'is_available': True,
 u'mileage': u'13000-13499',
 u'model': u'BA

In [27]:
import pandas as pd

In [28]:
df = pd.DataFrame(list(cars.find()))

In [29]:
df.county.unique()

array([u'Lund', u'Uppsala', u'Skaraborg', u'\xd6rebro', u'Malm\xf6',
       u'G\xf6teborg', u'Stockholm', u'V\xe4sterbotten', u'\xc4lvsborg',
       u'V\xe4stmanland', u'Kristianstad', u'J\xe4mtland', u'Halland',
       u'G\xe4vleborg', u'Kalmar', u'Dalarna', u'H\xe4ssleholm',
       u'Blekinge', u'Helsingborg', u'V\xe4rmland', u'\xd6sterg\xf6tland',
       u'J\xf6nk\xf6ping', u'V\xe4sternorrland', u'Kronoberg',
       u'Simrishamn', u'Osby', u'S\xf6dermanland', u'H\xf6rby',
       u'Lule\xe5', u'Pite\xe5', u'Gotland', u'\xc4ngelholm',
       u'Landskrona', u'Trelleborg', u'B\xe5stad', u'Klippan',
       u'H\xf6\xf6r', u'Arvidsjaur', u'Kalix', u'Ystad', u'Boden',
       u'\xd6verkalix', u'Kiruna', u'G\xe4llivare', u'Esl\xf6v',
       u'\xd6vertorne\xe5', u'Pajala', u'Haparanda', u'Jokkmokk',
       u'\xc4lvsbyn', u'Arjeplog'], dtype=object)

In [32]:
df.to_csv('blocket_cars.csv', encoding='utf-8', index_label=False)